### vanilla Rnn의 문제점

 - 긴 sequence를 가지면 vanishing gradient 발생
 - Back-prooagation 이 제대로 이루어지지 않는다.


## LSTM(Long Short Term Memory Network)
 - 기본 RNN에 구조에 cell state를 추가
 - cell state : 중요한 정보를 남기거나 추가하는 기능
 - cell state에는 현재의 정보와 다음에 넘겨줄 정보가 같이 있으니, hidden state($h_t$)에 현재의 정보를 가공해서 추출
 - $C_{t-1}$(cell state)의 정보들의 gate coefficient(중요도,$G$)를 구한다.
    -> using small non-linear layer 

- 그 후 $C_{t-1}*G $를 구하여 남길 정보를 추출
![lstm](./lstm.png)

- 과정

 1) $ C_{t-1}$에서 불필요한 정보제거 ($f_t = \sigma (W_f[h_{t-1},x_t] +b_f)$(forget gate))

 2) $ C_{t-1}$에 새로운 input $x_t$와 $h_{t-1}$을 보고 중요한 정보를 넣는다.(($i_t = \sigma (W_i[h_{t-1},x_t] +b_i)$(input gate), $C_t' = tanh(W_c[h_{t-1},x_t] + b_c$)

 3) 위과정을 통해 $C_t$를 만든다. ($C_t = f_t*C_{t-1} + i_t * C_t'$)

 4) $C_t$를 적절히 가공해 해당 t에서 $h_t$를 만든다.($h_t = o_t *tanh(C_t)$,$o_t = \sigma (W_o[h_{t-1},x_t] + b_o)$)

### vanishing gradient 가 발생하지 않는이유
 - cell state에 정보가 기록이 되고 나중에 지워지지 않는 다면,
 - non-linear activation function을 거치지 않고 t+n 시점까지 흘러오기 때문에 상당부분 해결가능


## GRU(Gated Recurrent Unit)
 - Lstm의 gate가 많은 문제를 해결하려고 했다.
 
 1) reset gate를 계산하여 임시 $h_t$를 만든다. ($r_t = \sigma(h_{t-1},x_t)$, $h_t' = tanh(W[r_t * h_{t-1},x_t])$)
 
 2) update gate를 통해 $h_{t-1}과 h_t'$간의 비중을 결정($z_t = \sigma(W_z[h_{t-1},x_t])$)

 3) $z_t$를 이용해 $h_t$를 계산한다.($h_t = (1-z_t) * h_{t-1} + z_t * h_t'$) 

In [5]:
import pandas as pd
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error

import seaborn as sns 
import matplotlib.pyplot as plt

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.8.1
Populating the interactive namespace from numpy and matplotlib


In [7]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_layer, batch_size, dropout,use_bn):
        
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.num_layer = num_layer
        
        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn
        
        self.lstm = nn.LSTM(self.in_dim, self.hid_dim, self.num_layer)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hid_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hid_dim))
    
    # 간단한 MLP model로 생각하면 됨.
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hid_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hid_dim, self.hid_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hid_dim // 2, self.hid_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        
        # lstm_out :마지막 레이어의 h(??), self.hidden : 모든 layer의 hidden_state
        lstm_out, self.hidden = self.lstm(x, self.hidden) 
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred